<a href="https://colab.research.google.com/github/gudals6676/Machine_Learning_Study/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 목표
- 네이버 영화리뷰 데이터셋을 이용해 긍정/부정 분류 모델을 만들어보자
- TF=IDF 방식을 적용해보자
- konlpy 한국어 형태소 분석기를 설치하고 활용해보자

In [3]:
!pip install Konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 11.6MB/s 
     |████████████████████████████████| 450kB 39.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


### 데이터수집
- 네이버 데이터를 로드
- label(0:부정, 1:긍정)

In [5]:
import pandas as pd

In [8]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/ratings_train.txt", delimiter="\t")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/ratings_test.txt", delimiter="\t")
# delimiter : 파일에서 데이터 사이의 간격을 구분해주는 구분자(설정하지 않으면, 로 구분)

In [11]:
# 컬럼의 너비를 최대치로 설정
pd.set_option("display.max_colwidth", -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [12]:
df_train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [23]:
df_test.head(10)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
5,7898805,"음악이 주가 된, 최고의 음악영화",1
6,6315043,진정한 쓰레기,0
7,6097171,"마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다",0
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다,0
9,6242223,"이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",1


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [17]:
# train, test 데이터의 결측치가 있는 행을 불리언 인덱싱으로 확인해보자잇~!
df_train[df_train["document"].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [18]:
df_test[df_test["document"].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [19]:
#결측치가 있는 행을 제거
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [22]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


- 문제/정답, 학습/평가 데이터 분리

In [28]:
X_train = df_train["document"]
y_train = df_train["label"]
X_test = df_test["document"]
y_test = df_test["label"]

In [30]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(149995,)
(49997,)
(149995,)
(49997,)


### BOW 기법(단어사전 만들기)
- 1. CounterVectorize(단순히 단어의 빈도만 파악)
- 2. TF-IDF(단어의 빈도+ 단어가 등장하는 문서의 개수 둘다 파악)


In [34]:
# TF-IDF를 판단하기 위해 TfidfVectorizer를 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
tf_idf_vect = TfidfVectorizer()
# TfidfVectorizer의 디폴트값은 word(단어)기준으로 토큰화를 진행

In [35]:
tf_idf_vect.fit(X_train[:3])
# TF-IDF 방식으로 단어사전 만들기(학습)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [37]:
# vocabulary_ : 토큰화 된 단어와 인덱스의 라벨을 확인
tf_idf_vect.vocabulary_

{'가볍지': 0,
 '너무재밓었다그래서보는것을추천한다': 1,
 '더빙': 2,
 '목소리': 3,
 '않구나': 4,
 '오버연기조차': 5,
 '진짜': 6,
 '짜증나네요': 7,
 '초딩영화줄': 8,
 '포스터보고': 9}

- 영어에 맞춰져 있어서 한글 텍스트에서는 토큰화가 제대로 이루어지고 있지 않음

### 한글에 특화되어 있는 konlpy - Kkma(꼬꼬마) 사용하기

In [38]:
# 한글을 형태소별로 잘 토큰화 할 수 있는 꼬꼬마 라이브러리를 불러오기
from konlpy.tag import Kkma

In [39]:
kkma = Kkma()

In [41]:
# nouns : 명사만 토큰화 시켜보자!
kkma.nouns(X_train[0])

['더빙', '목소리']

In [42]:
kkma.nouns("지윤씨의 이상형은 가슴털이 수북한 전현무 스타일!")

['지', '지윤씨의', '윤씨의', '이상형', '가슴', '가슴털', '털', '전', '전현무', '현무', '스타일']

In [43]:
# 입력 문장이 들어왔을때 명사만 토큰화 시키는 함수를 선언
def myTokenizer(text) :
  return kkma.nouns(text)

In [45]:
# kkma 라이브러리에 TF-IDF 방식을 활용해보자
# tokenizer : 토큰의 단위를 지정, 디폴트 값은 word(단어)
tf_idf_vect =TfidfVectorizer(tokenizer=myTokenizer)

# 3개의 행에 대해서만 먼저 진행해보자
tf_idf_vect.fit(X_train[:3])
tf_idf_vect.vocabulary_

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'더빙': 0,
 '딩': 1,
 '목소리': 2,
 '밓': 3,
 '보고': 4,
 '연기': 5,
 '영화': 6,
 '오버': 7,
 '오버연기': 8,
 '재': 9,
 '재밓': 10,
 '줄': 11,
 '초': 12,
 '초딩영화줄': 13,
 '추천': 14,
 '포스터': 15,
 '포스터보고': 16,
 '흠': 17}

### POS(Part of Speech) Tagging 활용
- 문장을 토큰화 한 후 쪼개진 형태소에 품사를 부여하는 것

In [47]:
# 한글에서는 형용사와 동사에 감정이 많이 들어가 있음
data = "먹는다 먹다 먹었다 이쁘다 아름답다 사진 모자"
# morphs : 텍스트(문장)를 형태소로 분리
kkma.morphs(data)

['먹', '는', '다', '먹', '다', '먹', '었', '다', '이쁘', '다', '아름답', '다', '사진', '모자']

In [48]:
# pos : 텍스트의 각 형태소에 품사 정보를 표시
kkma.pos(data)

[('먹', 'VV'),
 ('는', 'EPT'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('었', 'EPT'),
 ('다', 'EFN'),
 ('이쁘', 'VA'),
 ('다', 'ECS'),
 ('아름답', 'VA'),
 ('다', 'ECS'),
 ('사진', 'NNG'),
 ('모자', 'NNG')]

In [ ]:
kkma.tagset

In [ ]:
# 학습을 시키기 위해 데이터프레임화
df = pd.DataFrame(kkma.pos(data), columns=["morph", "tag"])
df

In [51]:
# 데이터프레임에서 tag를 인덱스로 설정
df.set_index("tag", inplace=True)
# 동사(VV), 형용사(VA), 보통명사(NNG)로 인덱싱
df.loc[["VV", "VA", "NNG"]]

,morph
tag,
VV,먹
VV,먹
VV,먹
VA,이쁘
VA,아름답
NNG,사진
NNG,모자


In [52]:
# 입력 텍스트에 대해서 pos tagging을 진행하여 VV, VA, NNG값만 반환하는 함수를 선언
def myTokenizer2(text) :
  # 텍스트가 들어오면 데이터프레임으로 변경 후 품사를 인덱스로 설정
  df = pd.DataFrame(kkma.pos(text), columns=["morph", "tag"])
  df.set_index("tag", inplace=True)

  # VV이거나 VA이거나 NNG일 때 그 값을 반환
  if ("VV" in df.index) | ("VA" in df.index) | ("NNG" in df.index):
    labels = ["VV", "VA", "NNG"]
    # intersection() : 괄호안의 라벨들의 교집합까지 포함해서 배열로 변환
    return df.loc[df.index.intersection(labels)]["morph"].values
    # VV, VA, NNG가 아니라면 필요없기 때문에 빈 배열로 반환
  else :
    return []


In [53]:
# 감정 분석에 필요한 동사, 형용사, 보통명사만 추출하는 TF-IDF 백터라이저를 생성
tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer2)
tf_idf_vect.fit(X_train[:2])
tf_idf_vect.vocabulary_

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'가볍': 0,
 '더빙': 1,
 '목소리': 2,
 '보고': 3,
 '아': 4,
 '연기': 5,
 '영화': 6,
 '오버': 7,
 '줄': 8,
 '짜증나': 9,
 '포스터': 10,
 '흠': 11}

In [54]:
final_tf_idf_vect = TfidfVectorizer(tokenizer=myTokenizer2)
final_tf_idf_vect.fit(X_train[:3000])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function myTokenizer2 at 0x7fc94c7f8830>,
                use_idf=True, vocabulary=None)

In [55]:
# 동사, 형용사, 일반명사만 추출하여 만들어진 단어모음의 개수
len(final_tf_idf_vect.vocabulary_)

4743

In [56]:
# 토큰화된 단어를 기계가 이해할 수 있도록 벡터형태로 수치화(transform)
X_train = final_tf_idf_vect.transform(X_train[:3000])
X_test = final_tf_idf_vect.transform(X_test[:3000])

In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
logi = LogisticRegression()
logi.fit(X_train, y_train[:3000])
logi.score(X_test, y_test[:3000])

0.7533333333333333